In [2]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
 
runnable=RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough().assign(mult=lambda x:x["num"]*3),
    modified=lambda x:x["num"]+1
 )
 
runnable.invoke({"num":1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

运行自定义函数
可以在管道中使用任意函数，这些函数必须是单个输入，如果有一个接受多个参数的函数，则应该编写一个包装函数，将其转换为单个输入函数。

In [3]:
from langchain_openai import OpenAI,ChatOpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.7
)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from operator import  itemgetter

In [5]:
from langchain_core.runnables import RunnableLambda

def length_function(text):
    return len(text)
  
def _multiple_length_function(text1,text2):
    return len(text1)*len(text2)
  
def multi_length_function(_dict):
    return _multiple_length_function(_dict["text1"],_dict["text2"])
  
prompt=ChatPromptTemplate.from_template("what is {a}+{b}")

chain1=prompt | model

chain=(
    {
        "a":itemgetter("foo") | RunnableLambda(length_function),
        "b":{"text1":itemgetter("foo"),"text2":itemgetter("bar")} | RunnableLambda(multi_length_function),
    }
    | prompt
    | model
)

In [6]:
chain.invoke({"foo":"bar","bar":"gah"})

AIMessage(content='The sum of 3 and 9 is 12.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 25, 'total_tokens': 38, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-coder-1.5b-instruct', 'system_fingerprint': 'qwen2.5-coder-1.5b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb9c65c1-4eb1-433d-898d-fc39176988b8-0', usage_metadata={'input_tokens': 25, 'output_tokens': 13, 'total_tokens': 38, 'input_token_details': {}, 'output_token_details': {}})

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate.from_template(
  """鉴于下面的用户问题，将其分类为'Langchain','OpenAI'或'其他'。
  
  不要用超过一个字来回应。
  
  <question>
  {question}
  </question>
  
  分类："""
)

chain=(
    prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question":"我想知道Langchain的文档"})

'Langchain'

In [9]:
langchainPrompt = PromptTemplate.from_template(
    """您是 langchain 方面的专家。
回答问题时始终以“正如老陈告诉我的那样”开头。
回答以下问题：

问题：{question}
回答："""
)
langchain_chain = langchainPrompt | model

In [10]:
OpenAIPrompt = PromptTemplate.from_template(
    """您是 OpenAI 方面的专家。
回答问题时始终以“正如奥特曼告诉我的那样”开头。
回答以下问题：

问题：{question}
回答："""
)
OpenAI_chain = OpenAIPrompt | model

In [11]:
generalPrompt = PromptTemplate.from_template(
    """回答以下问题：

问题：{question}
回答："""
)
general_chain=generalPrompt | model 

In [12]:
def route(info):
    if "OpenAI" in info["topic"]:
        return OpenAI_chain
    elif "Langchain" in info["topic"]:
        return langchain_chain
    else:
        return general_chain     

In [13]:
full_chain={"topic":chain,"question": lambda x: x["question"]} | RunnableLambda(route)

In [14]:
full_chain.invoke({"question":"Langchain的文档是怎么样的？"})

AIMessage(content='正如老陈告诉我那样，Langchain 的文档是非常详细的。你可以通过官方网站或 GitHub 上的官方仓库找到最新的文档和教程。此外，社区和论坛上也有大量的资源可以帮助你学习和使用 Langchain。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 58, 'total_tokens': 103, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-coder-1.5b-instruct', 'system_fingerprint': 'qwen2.5-coder-1.5b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-213ffe72-09d5-496f-ad73-fe95e060869a-0', usage_metadata={'input_tokens': 58, 'output_tokens': 45, 'total_tokens': 103, 'input_token_details': {}, 'output_token_details': {}})